In [2]:


import torch
import gc
import os
import warnings
warnings.filterwarnings("ignore")

def check_gpu_setup():
    """Check GPU availability and setup"""
    print("🔍 Checking GPU setup...")

    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"   ✅ GPU Found: {gpu_name}")
        print(f"   💾 GPU Memory: {gpu_memory:.1f} GB")

        # Clear GPU memory
        torch.cuda.empty_cache()
        gc.collect()

        return True, gpu_name
    else:
        print("   ❌ No GPU available")
        return False, "CPU"

def load_zeroscope_model_gpu():
    """Load ZeroScope model optimized for T4 GPU"""
    print("\n🚀 Loading ZeroScope Model for T4 GPU...")

    try:
        from diffusers import DiffusionPipeline
        import torch

        # Clear memory first
        torch.cuda.empty_cache()
        gc.collect()

        print("   📦 Loading model files...")

        # Load without fp16 variant (which doesn't exist for this model)
        pipe = DiffusionPipeline.from_pretrained(
            "cerspense/zeroscope_v2_576w",
            torch_dtype=torch.float16,  # Use half precision for GPU
            use_safetensors=False,      # Don't require safetensors
            low_cpu_mem_usage=True,     # Efficient loading
            device_map="auto",          # Auto device mapping
        )

        # Move to GPU
        pipe = pipe.to("cuda")

        # GPU optimizations
        pipe.enable_attention_slicing()
        pipe.enable_vae_slicing()

        # Enable memory efficient attention if available
        try:
            pipe.enable_xformers_memory_efficient_attention()
            print("   ✅ XFormers optimization enabled")
        except:
            print("   ⚠️  XFormers not available, using default attention")

        print("   ✅ ZeroScope model loaded successfully on GPU!")
        return pipe

    except Exception as e:
        print(f"   ❌ GPU model loading failed: {e}")
        print("   🔄 Trying CPU fallback...")
        return load_zeroscope_model_cpu()

def load_zeroscope_model_cpu():
    """Fallback CPU loading"""
    try:
        from diffusers import DiffusionPipeline
        import torch

        print("   📦 Loading model for CPU...")

        pipe = DiffusionPipeline.from_pretrained(
            "cerspense/zeroscope_v2_576w",
            torch_dtype=torch.float32,  # Float32 for CPU
            use_safetensors=False,
            low_cpu_mem_usage=True,
        )

        # CPU optimizations
        pipe.enable_attention_slicing()
        pipe.enable_sequential_cpu_offload()

        print("   ✅ ZeroScope model loaded on CPU!")
        return pipe

    except Exception as e:
        print(f"   ❌ CPU loading also failed: {e}")
        return None

def test_real_video_generation(pipe, device_name):
    """Test actual AI video generation"""
    if pipe is None:
        print("❌ No model available for testing")
        return False

    print(f"\n🎬 Testing REAL AI video generation on {device_name}...")

    try:
        prompt = "a beautiful butterfly landing on a colorful flower"
        print(f"   🎯 Prompt: '{prompt}'")

        # Generate video with optimal settings for T4
        with torch.no_grad():
            if torch.cuda.is_available():
                # GPU settings - higher quality
                video_frames = pipe(
                    prompt,
                    num_frames=12,           # More frames on GPU
                    height=320,              # Higher resolution
                    width=576,
                    num_inference_steps=15,  # Better quality
                    guidance_scale=9.0,
                ).frames[0]
            else:
                # CPU settings - faster generation
                video_frames = pipe(
                    prompt,
                    num_frames=8,
                    height=256,
                    width=320,
                    num_inference_steps=10,
                    guidance_scale=7.0,
                ).frames[0]

        print(f"   ✅ Generated {len(video_frames)} frames!")

        # Save the video
        import imageio
        video_filename = "real_ai_video.mp4"

        with imageio.get_writer(video_filename, fps=4) as writer:
            for frame in video_frames:
                writer.append_data(frame)

        print(f"   💾 Video saved as {video_filename}")

        # Display video info
        file_size = os.path.getsize(video_filename) / (1024 * 1024)
        print(f"   📊 Video size: {file_size:.1f} MB")

        return True

    except Exception as e:
        print(f"   ❌ Video generation failed: {e}")
        return False

def create_updated_video_app(model):
    """Create updated video app with real AI model"""
    print("\n🌐 Creating REAL AI Video App...")

    from fastapi import FastAPI, HTTPException
    from fastapi.responses import HTMLResponse, FileResponse
    from fastapi.middleware.cors import CORSMiddleware
    from pydantic import BaseModel
    import uuid
    from pathlib import Path
    import imageio

    # Create storage
    video_storage = Path("ai_videos")
    video_storage.mkdir(exist_ok=True)

    class VideoRequest(BaseModel):
        prompt: str
        duration: int = 3

    class VideoResponse(BaseModel):
        video_id: str
        status: str
        message: str
        video_url: str = None

    app = FastAPI(title="Real AI Video Generator")

    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    def generate_ai_video(prompt: str):
        """Generate real AI video"""
        try:
            print(f"🎬 Generating AI video: '{prompt[:50]}...'")

            with torch.no_grad():
                if torch.cuda.is_available():
                    # GPU generation - high quality
                    video_frames = model(
                        prompt,
                        num_frames=16,
                        height=320,
                        width=576,
                        num_inference_steps=20,
                        guidance_scale=9.0,
                    ).frames[0]
                else:
                    # CPU generation
                    video_frames = model(
                        prompt,
                        num_frames=12,
                        height=256,
                        width=320,
                        num_inference_steps=15,
                        guidance_scale=7.0,
                    ).frames[0]

            # Save video
            video_id = str(uuid.uuid4())
            video_path = video_storage / f"{video_id}.mp4"

            with imageio.get_writer(str(video_path), fps=4) as writer:
                for frame in video_frames:
                    writer.append_data(frame)

            file_size = os.path.getsize(video_path) / (1024 * 1024)
            print(f"✅ AI video generated: {video_id} ({file_size:.1f} MB)")

            return video_id, str(video_path)

        except Exception as e:
            print(f"❌ AI video generation failed: {e}")
            raise HTTPException(status_code=500, detail=f"Generation failed: {str(e)}")

    @app.get("/", response_class=HTMLResponse)
    async def root():
        device_info = "🚀 T4 GPU" if torch.cuda.is_available() else "💻 CPU"
        return f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>🎬 Real AI Video Generator</title>
            <style>
                body {{ font-family: Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; }}
                .container {{ background: rgba(255,255,255,0.95); padding: 40px; border-radius: 20px; box-shadow: 0 25px 50px rgba(0,0,0,0.15); }}
                .status {{ background: linear-gradient(135deg, #28a745, #20c997); color: white; padding: 20px; border-radius: 12px; margin: 25px 0; text-align: center; font-weight: bold; font-size: 18px; }}
                h1 {{ color: #333; text-align: center; margin-bottom: 15px; font-size: 2.5em; }}
                textarea {{ width: 100%; height: 120px; padding: 20px; border: 3px solid #ddd; border-radius: 12px; font-size: 16px; resize: vertical; box-sizing: border-box; }}
                button {{ width: 100%; padding: 20px; background: linear-gradient(45deg, #667eea, #764ba2); color: white; border: none; border-radius: 12px; font-size: 20px; cursor: pointer; margin-top: 20px; font-weight: bold; }}
                button:hover {{ opacity: 0.9; transform: translateY(-2px); }}
                button:disabled {{ opacity: 0.6; cursor: not-allowed; transform: none; }}
                .loading {{ display: none; text-align: center; margin: 30px 0; }}
                .spinner {{ border: 5px solid #f3f3f3; border-top: 5px solid #667eea; border-radius: 50%; width: 60px; height: 60px; animation: spin 1s linear infinite; margin: 0 auto; }}
                @keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}
                video {{ width: 100%; max-width: 500px; border-radius: 12px; margin-top: 25px; box-shadow: 0 10px 25px rgba(0,0,0,0.15); }}
                .result {{ display: none; text-align: center; margin-top: 25px; }}
                .error {{ background: #f8d7da; color: #721c24; padding: 20px; border-radius: 12px; margin: 25px 0; }}
                .examples {{ background: #f8f9fa; padding: 20px; border-radius: 12px; margin: 20px 0; }}
                .examples h3 {{ margin-top: 0; color: #495057; }}
                .examples ul {{ text-align: left; }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>🎬 Real AI Video Generator</h1>
                <div class="status">🤖 REAL AI MODEL LOADED - {device_info}</div>

                <div class="examples">
                    <h3>💡 Example Prompts (try these!):</h3>
                    <ul>
                        <li><strong>Nature:</strong> "A butterfly landing on a sunflower"</li>
                        <li><strong>Abstract:</strong> "Colorful paint drops splashing in water"</li>
                        <li><strong>Animals:</strong> "A cute cat playing with yarn"</li>
                        <li><strong>Scenery:</strong> "Ocean waves crashing on a beach at sunset"</li>
                        <li><strong>Weather:</strong> "Rain drops falling on green leaves"</li>
                    </ul>
                </div>

                <form id="videoForm">
                    <textarea id="prompt" placeholder="Describe your AI-generated video:

🎨 Be descriptive and creative!
🎬 The AI will generate a unique video based on your prompt
⚡ Generation takes 2-5 minutes on GPU" required></textarea>
                    <button type="submit" id="generateBtn">🚀 Generate Real AI Video</button>
                </form>

                <div class="loading" id="loading">
                    <div class="spinner"></div>
                    <p><strong>🤖 AI is generating your video... ⏳</strong></p>
                    <p>This may take 2-5 minutes for real AI generation</p>
                    <p><em>Creating unique video frames based on your prompt...</em></p>
                </div>

                <div class="result" id="result">
                    <h3>🎉 Your AI video is ready!</h3>
                    <video id="generatedVideo" controls autoplay muted loop></video>
                    <p><strong>🤖 Generated with real AI model!</strong></p>
                </div>
            </div>

            <script>
                document.getElementById('videoForm').addEventListener('submit', async function(e) {{
                    e.preventDefault();

                    const prompt = document.getElementById('prompt').value;
                    if (!prompt.trim()) {{
                        alert('Please enter a video description');
                        return;
                    }}

                    // Show loading
                    document.getElementById('generateBtn').disabled = true;
                    document.getElementById('loading').style.display = 'block';
                    document.getElementById('result').style.display = 'none';

                    // Clear errors
                    document.querySelectorAll('.error').forEach(e => e.remove());

                    try {{
                        const response = await fetch('/generate-video', {{
                            method: 'POST',
                            headers: {{ 'Content-Type': 'application/json' }},
                            body: JSON.stringify({{ prompt: prompt, duration: 3 }})
                        }});

                        const data = await response.json();

                        if (data.status === 'success') {{
                            document.getElementById('generatedVideo').src = data.video_url + '?t=' + Date.now();
                            document.getElementById('result').style.display = 'block';
                        }} else {{
                            const errorDiv = document.createElement('div');
                            errorDiv.className = 'error';
                            errorDiv.innerHTML = '<strong>Error:</strong> ' + data.message;
                            document.getElementById('loading').after(errorDiv);
                        }}
                    }} catch (error) {{
                        const errorDiv = document.createElement('div');
                        errorDiv.className = 'error';
                        errorDiv.innerHTML = '<strong>Network Error:</strong> ' + error.message;
                        document.getElementById('loading').after(errorDiv);
                    }} finally {{
                        document.getElementById('generateBtn').disabled = false;
                        document.getElementById('loading').style.display = 'none';
                    }}
                }});
            </script>
        </body>
        </html>
        """

    @app.post("/generate-video", response_model=VideoResponse)
    async def generate_video_endpoint(request: VideoRequest):
        try:
            prompt = request.prompt.strip()
            if not prompt:
                raise HTTPException(status_code=400, detail="Prompt cannot be empty")

            video_id, video_path = generate_ai_video(prompt)

            return VideoResponse(
                video_id=video_id,
                status="success",
                message="Real AI video generated successfully!",
                video_url=f"/video/{video_id}"
            )

        except Exception as e:
            return VideoResponse(
                video_id="",
                status="error",
                message=str(e)
            )

    @app.get("/video/{video_id}")
    async def get_video(video_id: str):
        video_path = video_storage / f"{video_id}.mp4"
        if not video_path.exists():
            raise HTTPException(status_code=404, detail="Video not found")
        return FileResponse(str(video_path), media_type="video/mp4")

    @app.get("/health")
    async def health():
        return {
            "status": "healthy",
            "device": "cuda" if torch.cuda.is_available() else "cpu",
            "model_loaded": True,
            "gpu_memory": f"{torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else "N/A"
        }

    print("   ✅ Real AI Video App created!")
    return app

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    print("🎬 REAL AI VIDEO GENERATOR - T4 GPU VERSION")
    print("=" * 60)

    # Check GPU
    has_gpu, device_name = check_gpu_setup()

    # Load model
    if has_gpu:
        model = load_zeroscope_model_gpu()
    else:
        model = load_zeroscope_model_cpu()

    if model is None:
        print("❌ Failed to load any model")
        return

    # Test generation
    success = test_real_video_generation(model, device_name)

    if not success:
        print("❌ Video generation test failed")
        return

    # Create and run app
    app = create_updated_video_app(model)

    print("\n" + "=" * 60)
    print("🎉 REAL AI VIDEO GENERATOR IS READY!")
    print("=" * 60)
    print("🤖 Status: Real AI Model Loaded")
    print(f"💻 Device: {device_name}")
    print("⚡ Ready for high-quality AI video generation!")

    # Start server
    import uvicorn
    import threading

    def run_server():
        uvicorn.run(app, host="127.0.0.1", port=8001, log_level="error")

    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    import time
    time.sleep(3)

    # Get public URL
    try:
        from google.colab.output import eval_js
        public_url = eval_js("google.colab.kernel.proxyPort(8001)")
        print(f"🌐 Public URL: {public_url}")

        from IPython.display import display, HTML
        display(HTML(f'''
        <div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); padding: 30px; border-radius: 20px; margin: 20px 0; color: white;">
            <h2 style="margin: 0 0 20px 0;">🎉 REAL AI Video Generator Ready!</h2>
            <p><strong>✅ Real ZeroScope AI model loaded on {device_name}!</strong></p>
            <div style="margin: 25px 0;">
                <a href="{public_url}" target="_blank" style="display: inline-block; background: rgba(255,255,255,0.2); color: white; padding: 15px 30px; text-decoration: none; border-radius: 10px; font-weight: bold; border: 2px solid white;">
                    🚀 Open AI Video Generator
                </a>
            </div>
            <p style="margin: 15px 0; font-size: 16px;"><strong>URL:</strong> {public_url}</p>
        </div>
        '''))

    except:
        public_url = "http://localhost:8001"
        print(f"🔗 Local URL: {public_url}")

    print("\n⏰ Server running... Generate some AI videos!")

# Run it
main()

🎬 REAL AI VIDEO GENERATOR - T4 GPU VERSION
🔍 Checking GPU setup...
   ✅ GPU Found: Tesla T4
   💾 GPU Memory: 15.8 GB

🚀 Loading ZeroScope Model for T4 GPU...
   📦 Loading model files...
   ❌ GPU model loading failed: auto not supported. Supported strategies are: balanced
   🔄 Trying CPU fallback...
   📦 Loading model for CPU...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

   ✅ ZeroScope model loaded on CPU!

🎬 Testing REAL AI video generation on Tesla T4...
   🎯 Prompt: 'a beautiful butterfly landing on a colorful flower'


  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ Generated 12 frames!


   💾 Video saved as real_ai_video.mp4
   📊 Video size: 0.2 MB

🌐 Creating REAL AI Video App...
   ✅ Real AI Video App created!

🎉 REAL AI VIDEO GENERATOR IS READY!
🤖 Status: Real AI Model Loaded
💻 Device: Tesla T4
⚡ Ready for high-quality AI video generation!
🌐 Public URL: https://8001-gpu-t4-s-3ks5bc5x31gin-a.us-west4-1.prod.colab.dev



⏰ Server running... Generate some AI videos!


In [3]:
!nvidia-smi


Sat Aug 16 06:08:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             46W /   70W |    1580MiB /  15360MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----